<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Concentration_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Concentration calculation
This notebook is used to calculate the concentration of phtalates in an environemental sample using an internal standard for quantitation. The Calibration_calculation.ipynb notebook, yields severall of the required files nescessary for the calculations in this notebook. The files *Cal_{n}_analyte_linreg.csv* need to be used which is located in the *calibration_output* folder on the google drive. If this notebook is not applied from my machine in connection with my google drive, either the path to the required files needs to be adabted or changed.
The following code allows for the calculation of the concentration, by using the linear regression form a calibration series and the ratio of the signal of the analyte with the internal standard (area). All the data regarding the calibration are present on my google drive in the folders: *calibration_input* and *calibration_output*, while a new google drive folder will be used to save the final data in as well as a new input folder.

*Google drive folder:*
quantitation_input
quantitation_output

The .csv files containing the peak list from the sample should be saved in the in the input folder. The names should follow a consequent naming sceme:

***Sample_{date_of_analysis}_{tag_bonus_info}_{Sample_Nr}_pl.csv***

Here the Sample number *(Sample_Nr)* is of key importance and needs to be properly documented in the Labjournal, as it is the main way of differentiation.


The following code is used to import all the nescessary libraries and mount the google drive.

In [24]:
import numpy as np
from google.colab import drive
import glob
import pandas as pd
import os
import csv
from datetime import datetime
import re
import ast

drive.mount('/content/drive', force_remount = True)

sample_nr_low = 3 #needs to be addapted to 1 nr in the series
sample_nr_high = 26 #needs to be addapted to the last nr of the series

Mounted at /content/drive


The following code is the addapted code from the Calibration_calculation.ipynb and yields the separated in separated .csv files, which will be used to calculate the ratio in a following code segment.

The output file has the following filename and is saved in the quantitation_output folder:

***Sample_{n}_row{i+1}.csv***

In [25]:
#Loop over all the designated sample files.
for n in range(sample_nr_low, sample_nr_high):
    files = glob.glob(f'/content/drive/My Drive/quantitation_input/*_{n}_pl.csv')

    if not files:
        print(f'Files matching pattern Sample Nr. {n} was not found.')
        continue

    else:
        # Loop over each row and save a separate data frame for each
        df = pd.read_csv(files[0], header=4)
        num_rows = len(df)

        for i in range(num_rows):
            row_dfs = []
            for file in files:
                df = pd.read_csv(file, header=4, na_values='')

                # Extract the i'th row of the CSV file and append it to the row data frame list
                row = df.iloc[i, :].dropna().astype(str)
                row_dfs.append(row)

            # Concatenate the row data frames along the columns axis and convert to a data frame
            combined_df = pd.concat(row_dfs, axis=1).T

            # Save the combined data frame to a CSV file with a name indicating the row number
            combined_df.to_csv(f'/content/drive/My Drive/quantitation_output/Sample_{n}_row{i+1}.csv', index=False, header=False)

            # Convert all values to strings before joining and print the row in the desired format
            formatted_row = '\t'.join(map(str, combined_df.values[0]))
            print(formatted_row)

13.85	13.79	13.92	21071172602.484	100.0	10074772322.069	100.0
15.78	15.73	15.85	22407607845.484	13.44	10613189528.744	14.02
18.63	18.58	18.69	25045005022.938	100.0	11827144359.784	100.0
20.64	20.58	20.71	42791053983.488	25.67	21595775236.589	28.54
25.65	25.59	25.71	21577714528.596	12.94	10538747626.577	13.93
26.78	26.73	26.86	35316264158.461	21.18	17483405756.132	23.1
29.32	29.25	29.44	44629782353.569	26.77	15448325561.527	20.41
13.83	13.76	13.97	2555108.643	100.0	1164212.394	100.0
15.77	15.66	15.95	8511470.846	16.13	4090371.89	17.04
18.62	18.58	18.69	210187203.541	100.0	98901778.104	100.0
20.63	20.48	20.83	18360409.883	34.8	8813832.161	36.71
25.64	25.6	25.69	2342839.694	4.44	1087930.381	4.53
26.78	26.72	26.92	20693055.33	39.23	9152849.827	38.13
29.32	29.23	29.38	2844987.325	5.39	862264.766	3.59
13.83	13.75	13.97	1844472.025	100.0	858155.593	100.0
15.77	15.73	15.84	9162894.193	19.15	4359022.79	19.53
18.62	18.58	18.69	256539593.1	100.0	120889015.362	100.0
20.63	20.58	20.72	17329133.169	

The following code allows to calculate the ratio of the analytes with the internal standard. The output is saved as:

***Sample_{n}_row*_ratio.csv**

In [26]:
# Loop over all the designated sample files. Currently works but i am not sure if it might be just exactly the same as the first one, it works but not iterative...
for n in range(sample_nr_low, sample_nr_high):
    divisor_file = f'/content/drive/My Drive/quantitation_output/Sample_{n}_row3.csv'
    directory = '/content/drive/My Drive/quantitation_output'

    if not os.path.isfile(divisor_file):
        print(f"Divisor file does not exist: {divisor_file}")
        continue  # Skip to the next iteration if the divisor file is missing

    divisor_df = pd.read_csv(divisor_file, sep=',', header=None)
    csv_files = glob.glob(os.path.join(directory, f'*{n}_row*.csv'))

    # Process each CSV file
    for csv_file in csv_files:
        if csv_file == divisor_file or '_ratio' in csv_file:
            continue  # Skip the divisor file itself and ratio files
        data_df = pd.read_csv(csv_file, sep=',', header=None)
        data_df = data_df.apply(pd.to_numeric, errors='coerce')  # Convert values to numeric
        print('divisor_df:', divisor_df)
        print('data_df:', data_df)
        divided_data_df = pd.DataFrame()

        for i in range(len(data_df)):
          # Perform division on the data on the complete dataframe
          divided_row = data_df.iloc[i] / divisor_df.iloc[i]
          divided_data_df = divided_data_df.append(divided_row, ignore_index=True)

        # Create a new file name with the "_ratio" tag
        new_file_name = os.path.splitext(csv_file)[0] + '_ratio.csv'

        divided_data_df.to_csv(new_file_name, sep=',', header=False, index=False)
        control_import_df = pd.read_csv(new_file_name, sep=',', header=None)

        # Compare the recalculated values with the content of the ratio file
        """if control_import_df.equals(divided_data_df):
            print(f"The files match: {new_file_name}")
        else:
            print(f"The files do not match: {new_file_name}")"""

        #print(control_import_df)
        print(divided_data_df)
        print(csv_file)
        print('****************************************************')
    print('===========================================================')

divisor_df:        0      1      2             3      4             5      6
0  18.63  18.58  18.69  2.504501e+10  100.0  1.182714e+10  100.0
data_df:        0      1      2             3      4             5      6
0  13.85  13.79  13.92  2.107117e+10  100.0  1.007477e+10  100.0
          0         1         2         3    4         5    6
0  0.743425  0.742196  0.744783  0.841332  1.0  0.851835  1.0
/content/drive/My Drive/quantitation_output/Sample_3_row1.csv
****************************************************
divisor_df:        0      1      2             3      4             5      6
0  18.63  18.58  18.69  2.504501e+10  100.0  1.182714e+10  100.0
data_df:        0      1      2             3      4             5      6
0  15.78  15.73  15.85  2.240761e+10  13.44  1.061319e+10  14.02
          0         1         2         3       4         5       6
0  0.847021  0.846609  0.848047  0.894694  0.1344  0.897359  0.1402
/content/drive/My Drive/quantitation_output/Sample_3_row2.csv
*

<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a

          0         1         2         3      4         5       6
0  0.847021  0.846609  0.846442  0.000361  0.078  0.000366  0.0785
/content/drive/My Drive/quantitation_output/Sample_13_row2.csv
****************************************************
divisor_df:        0      1      2             3      4             5      6
0  18.63  18.58  18.69  2.504501e+10  100.0  1.182714e+10  100.0
data_df:        0      1      2             3      4             5      6
0  18.63  18.58  18.69  1.521248e+10  100.0  7.365030e+09  100.0
     0    1    2         3    4         5    6
0  1.0  1.0  1.0  0.607406  1.0  0.622723  1.0
/content/drive/My Drive/quantitation_output/Sample_13_row3.csv
****************************************************
divisor_df:        0      1      2             3      4             5      6
0  18.63  18.58  18.69  2.504501e+10  100.0  1.182714e+10  100.0
data_df:        0      1      2             3      4             5      6
0  20.63  20.57  20.71  3.560929e+07  30.69

<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a

          0         1         2        3      4         5       6
0  1.107354  1.107643  1.106474  0.00096  0.148  0.001008  0.1522
/content/drive/My Drive/quantitation_output/Sample_23_row4.csv
****************************************************
divisor_df:        0      1      2             3      4             5      6
0  18.63  18.58  18.69  2.504501e+10  100.0  1.182714e+10  100.0
data_df:        0     1      2            3     4            5     6
0  25.64  25.6  25.66  2853540.757  1.76  1344413.484  1.72
          0         1         2         3       4         5       6
0  1.376275  1.377826  1.372927  0.000114  0.0176  0.000114  0.0172
/content/drive/My Drive/quantitation_output/Sample_23_row5.csv
****************************************************
divisor_df:        0      1      2             3      4             5      6
0  18.63  18.58  18.69  2.504501e+10  100.0  1.182714e+10  100.0
data_df:        0      1      2             3      4           5      6
0  26.78  26.72

...

<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a

       0      1      2            3      4            5      6
0  13.83  13.79  13.89  5093579.669  100.0  2559012.761  100.0
         0         1         2         3    4         5    6
0  0.74275  0.741797  0.744772  0.101517  1.0  0.098645  1.0
/content/drive/My Drive/quantitation_output/Sample_23_row1.csv
****************************************************
divisor_df:        0      1      2             3      4             5      6
0  18.62  18.59  18.65  5.017472e+07  100.0  2.594156e+07  100.0
data_df:        0      1      2             3      4             5      6
0  15.77  15.72  15.85  4.651449e+07  28.65  2.346327e+07  29.97
          0         1         2        3       4         5       6
0  0.846939  0.845616  0.849866  0.92705  0.2865  0.904466  0.2997
/content/drive/My Drive/quantitation_output/Sample_23_row2.csv
****************************************************
divisor_df:        0      1      2             3      4             5      6
0  18.62  18.59  18.65  5.01

<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a

          0         1         2         3       4        5       6
0  0.847021  0.845699  0.848257  0.383818  0.3026  0.32979  0.3104
/content/drive/My Drive/quantitation_output/Sample_24_row2.csv
****************************************************
divisor_df:        0     1      2             3      4             5      6
0  18.63  18.6  18.65  1.766473e+07  100.0  1.022641e+07  100.0
data_df:        0      1      2            3      4            5      6
0  20.63  20.59  20.68  3379314.455  15.08  1719215.305  15.82
          0         1         2         3       4         5       6
0  1.107354  1.106989  1.108847  0.191303  0.1508  0.168115  0.1582
/content/drive/My Drive/quantitation_output/Sample_24_row4.csv
****************************************************
divisor_df:        0     1      2             3      4             5      6
0  18.63  18.6  18.65  1.766473e+07  100.0  1.022641e+07  100.0
data_df:        0      1      2           3     4          5     6
0  25.64  25.62 

<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-26-61a18a

The following code takes the ratio files as well as the parameters for the linear regression (analyte specific), and yields a final output file that contains all the information, such as the information of the ratio used for the quantitaiton, the parameters for the linear equation as well as other information such as compound id and other general sample information.

The final outpur files have the following name format:

***Sample_{n}_quantitation_output.csv***


In [27]:
current_date = datetime.today().strftime('%Y-%m-%d')
output_dir = '/content/drive/My Drive/quantitation_export/'
#header_row1 = [f'Analysis Nr. {n}', 'Phthalate concentration calculation', 'Concentration_calibration.ipynb', 'Till Vollmer', current_date]
header_row2 = ['Analyte', 'Intercept', 'Slope', 'Value', 'Concentration [microg/ml]']

row_descriptors = ['place holder','Dimethylphthalate', 'Diethylphthalate', 'Internal standard', 'Dibutylphthalate', 'Benzylbutylphthalate', 'Bis(2-ethyl-hexyl)phthalate', 'Di-n-octylphthalate']
file_pattern = '/content/drive/My Drive/quantitation_output/Sample_{}_row{}_ratio.csv'

for n in range(sample_nr_low, sample_nr_high):
    output_file = output_dir + f'Sample_{n}_concentration_output_{current_date}.csv'
    header_row1 = [f'Analysis Nr. {n}', 'Phthalate concentration calculation', 'Concentration_calibration.ipynb', 'Till Vollmer', current_date]

    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header_row1)
        writer.writerow(header_row2)

        for i in range(1, 8):
            if i == 3:
                continue
            print(f'Sample nr: {n}')
            print(f'Compound iteration: {i}')
            linreg_df = pd.read_csv(f'/content/drive/My Drive/calibration_export/Cal_{i}_analyte_linreg.csv')
            print(f'linear function information: {linreg_df}')
            descriptor = row_descriptors[i]
            print(f'Descriptor: {descriptor}')
            file_path = file_pattern.format(n, i)
            if not os.path.isfile(file_path):
                print(f"File not found for Sample {n}, Row {i}")
                continue

            sample_df = pd.read_csv(file_path, header=None, delimiter=',')
            print(f'Sample dataframe: {sample_df}')

            # Extracting the slope value from the linear function information
            intercept_str = str(linreg_df['Intercept'].values[0])
            slope_str = linreg_df['Slope'].values[0]

            # Remove the brackets before using regular expressions
            slope_str_cleaned = slope_str.strip('[]')
            slope_value = ast.literal_eval(slope_str_cleaned)
            if isinstance(slope_value, list):
                slope_value = slope_value[0]  # If slope is a list, take the first element

            intercept = float(re.findall(r'[-\d.]+', intercept_str)[0])
            slope = float(slope_value)

            fourth_column = sample_df.iloc[:, 3]  # Accessing the 4th column
            value = float(fourth_column.iloc[0])  # Assuming you want the value from the first row (index 0)
            concentration = slope * value + intercept
            print(f"Calculated concentration: {concentration}")
            row_data = [descriptor, intercept, slope, value, concentration]
            writer.writerow(row_data)

            # Additional print statements for debugging
            print(f"Sample DataFrame shape: {sample_df.shape}")
            print(f"Linreg DataFrame shape: {linreg_df.shape if linreg_df is not None else None}")
            print("------------------------------------------------------------------------")



Sample nr: 3
Compound iteration: 1
linear function information:    Intercept        Slope  R2 Value
0  -0.021661  [0.3460348]  0.961538
Descriptor: Dimethylphthalate
Sample dataframe:           0         1         2         3    4         5    6
0  0.743425  0.742196  0.744783  0.841332  1.0  0.851835  1.0
Calculated concentration: 0.26946918113532575
Sample DataFrame shape: (1, 7)
Linreg DataFrame shape: (1, 3)
------------------------------------------------------------------------
Sample nr: 3
Compound iteration: 2
linear function information:    Intercept         Slope  R2 Value
0  -0.027085  [0.39047037]  0.959185
Descriptor: Diethylphthalate
Sample dataframe:           0         1         2         3       4         5       6
0  0.847021  0.846609  0.848047  0.894694  0.1344  0.897359  0.1402
Calculated concentration: 0.32226616361103394
Sample DataFrame shape: (1, 7)
Linreg DataFrame shape: (1, 3)
------------------------------------------------------------------------
Sample nr